In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [123]:
from google.colab import drive
import os
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/IE7275/project/data'
file_path_holidays = os.path.join(folder_path, 'holidays_events.csv')
file_path_oil = os.path.join(folder_path, 'oil.csv')
file_path_stores = os.path.join(folder_path, 'stores.csv')
file_path_test = os.path.join(folder_path, 'test.csv')
file_path_train = os.path.join(folder_path, 'train.csv')
file_path_transactions = os.path.join(folder_path, 'transactions.csv')

df_holidays = pd.read_csv(file_path_holidays)
df_oil = pd.read_csv(file_path_oil)
df_stores = pd.read_csv(file_path_stores)
df_test = pd.read_csv(file_path_test)
df_train = pd.read_csv(file_path_train)
df_transactions = pd.read_csv(file_path_transactions)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [124]:
# Fill in the missing values ​​in the 'oil' column.
# method: backfill
print(df_oil.isnull().mean())
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate(method='linear').bfill()
print(df_oil.isnull().mean())

date          0.000000
dcoilwtico    0.035304
dtype: float64
date          0.0
dcoilwtico    0.0
dtype: float64


In [125]:
def add_date(df):
    print("in add_date")
    # 1. 确保日期格式正确
    df['date'] = pd.to_datetime(df['date'])

    # 2. 创建一个完整的索引 (日期 x 商店 x 类别), 有的日期可能没有记录从而导致缺失，通过这个方法补齐日期
    mux = pd.MultiIndex.from_product([
       pd.date_range(df['date'].min(), df['date'].max(), freq='D'),
       df['store_nbr'].unique(),
       df['family'].unique()
    ], names=['date', 'store_nbr', 'family'])

    # 3. 重新索引并补全 0
    # 这一步会自动补齐缺失日期，且不会像 unstack 那样搞乱列结构
    df = df.set_index(['date', 'store_nbr', 'family']).reindex(mux, fill_value=0).reset_index()
    return df

In [126]:
def left_join_oil(df):
   print('join oil')
   # left join: make oil join tran's df
   # make the oil table is the datetime type
   df_oil['date'] = pd.to_datetime(df_oil['date'])
   #Left Join
   df = df.merge(df_oil, on='date', how='left')
   return df

In [127]:
def left_join_stores(df):
  print('join_stores')
  # left join store
  df = df.merge(df_stores, on='store_nbr', how='left')
  return df

In [128]:
def left_join_holidays(df):

  print('join_holidays')
  # 1. 核心修改：先过滤掉被转移走的“假”假期，并保留补假（Transfer）和桥接日（Bridge）
  # 只保留真正产生放假效应的行
  real_holidays = df_holidays[df_holidays['transferred'] == False].copy()

  # 2. 合并逻辑（这里假设你已经处理好了 city/state 的匹配，建议用左连接）
  # 建议在合并前只保留需要的列，避免 train 变得臃肿
  real_holidays['date'] = pd.to_datetime(real_holidays['date'])
  df = df.merge(
      real_holidays[['date', 'type', 'locale', 'locale_name']],
      left_on = ['date','city'],
      right_on = ['date','locale_name'],
      how='left'
  )

  # 3. 创建布尔特征：如果 type 是 NaN，说明那天不是任何形式的假期
  df['if_holiday'] = np.where(df['type_y'].isna(), 0, 1)

  # 4. 进阶：处理“补班日”（Work Day）
  # 在数据集中，type='Work Day' 的日子虽然在假日表里，但其实要上班
  # 如果你想更精准，可以把 Work Day 重新标记为 0
  df.loc[df['type_y'] == 'Work Day', 'if_holiday'] = 0

  #5.去除无效列
  df = df.drop(columns = ['type_y','locale','locale_name'])
  return df

厄瓜多尔政府每月15日和最后一天发工资
加发薪日能帮助模型理解钱从哪里来。对于零售业预测，这通常是仅次于“促销”和“星期几”的第三大重要特征。

In [129]:
def join_payday(df):

  print('join_payday')
  df['is_payday'] = ((df['date'].dt.day == 15) | (df['date'].dt.is_month_end)).astype(int)

  return df

In [130]:
def rename(df):

  print('rename')
  df = df.rename(columns={'type_x': 'store_type', 'cluster': 'store_group'})

  return df

In [131]:
def join_earthquake(df):
  print('join_earthquake')
  # 假设你的 dataframe 叫 train
  # 厄瓜多尔大地震日期：2016-04-16
  earthquake_date = pd.to_datetime('2016-04-16')

  # 定义影响时长（比如震后 21 天）
  df['date'] = pd.to_datetime(df['date'])
  df['is_earthquake_period'] = df['date'].between(
      earthquake_date,
      earthquake_date + pd.Timedelta(days=21)
  ).astype(int)

  return df


In [132]:
def create_time_features(df):
    print('time_features')
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek  # 周几 (0-6)
    df['month'] = df['date'].dt.month           # 月份
    df['year'] = df['date'].dt.year             # 年份
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int) # 是否周末
    return df

In [133]:
def left_join_transactions(df):
  print('join_transactions')
  # 假设你已经读取了 train_df 和 transactions_df
  # 1. 确保日期格式一致
  df['date'] = pd.to_datetime(df['date'])
  df_transactions['date'] = pd.to_datetime(df_transactions['date'])

  # 2. 合并：按日期和商店编号
  # how='left' 保证不丢失训练集的行
  df = df.merge(df_transactions, on=['date', 'store_nbr'], how='left')

  # 3. 填充缺失值
  # 交易量缺失通常意味着商店关门或没生意，填 0
  df['transactions'] = df['transactions'].fillna(0)
  return df

In [134]:
def create_lag_features_sales(df):
    print('lag_features_sales')
    df = df.sort_values(['store_nbr', 'family', 'date']).reset_index(drop=True)
    # 按商店和产品系列分组
    grouped_sales = df.groupby(['store_nbr', 'family'])['sales']
    grouped_trans = df.groupby(['store_nbr', 'family'])['transactions']

    # 滞后特征 (为了安全，从16天前开始取值)
    df['lag_16_sales'] = grouped_sales.shift(16)
    df['lag_21_sales'] = grouped_sales.shift(21) # 上上周同期
    df['lag_28_sales'] = grouped_sales.shift(28) # 约一个月前

    # 滑动窗口均值 (过去一周的平均表现，同样偏移16天)
    df['rolling_mean_7_sales'] = grouped_sales.shift(16).rolling(window=7).mean().reset_index(0, drop=True)

    df['lag_16_trans'] = grouped_trans.shift(16)
    df['lag_21_trans'] = grouped_trans.shift(21) # 上上周同期
    df['lag_28_trans'] = grouped_trans.shift(28) # 约一个月前

    # 滑动窗口均值 (过去一周的平均表现，同样偏移16天)
    df['rolling_mean_7_trans'] = grouped_trans.shift(16).rolling(window=7).mean().reset_index(0, drop=True)
    df.fillna(0, inplace=True)

    return df

In [135]:
def switch_type(df):
  print('switch_type')
  # 转换col的type类型 防止内存爆炸
  cols_int = ['store_nbr', 'id', 'onpromotion', 'store_group', 'if_holiday','is_payday','is_earthquake_period','is_weekend']
  cols_float = ['sales', 'dcoilwtico','transactions','lag_16_sales','lag_21_sales','lag_28_sales','rolling_mean_7_sales','lag_16_trans','lag_21_trans','lag_28_trans','rolling_mean_7_trans']

  # 将指定的列转换为对应类型
  df[cols_int] = df[cols_int].astype('int32')
  df[cols_float] = df[cols_float].astype('float32')

  return df


In [138]:
def data_clean_train(df):
    df = add_date(df)
    df = left_join_oil(df)
    df = left_join_stores(df)
    df = left_join_holidays(df)
    df = join_payday(df)
    df = rename(df)
    df = join_earthquake(df)
    df = create_time_features(df)
    df = left_join_transactions(df)
    df = create_lag_features_sales(df)
    df = switch_type(df)
    return df

def data_clean_test(df):
    df = add_date(df)
    df = left_join_oil(df)
    df = left_join_stores(df)
    df = left_join_holidays(df)
    df = join_payday(df)
    df = rename(df)
    df = join_earthquake(df)
    df = create_time_features(df)
    # 不跑 transactions 和 lag_features，因为 test 没有 sales
    return df

# 3. 按照逻辑重新执行
train = data_clean_train(df_train) # 这里的 train 已经完全处理好
test = data_clean_test(df_test) # 让 test 也有 city, store_type 等

# 4. 拼接 (此时两边的列名应该是对齐的了)
train_tail = train.sort_values('date').tail(40 * 54 * 33)
df_combined = pd.concat([train_tail, test], axis=0).reset_index(drop=True)

# 5. 只对拼接后的数据补齐 Lag 和 Transaction 滞后
df_combined = create_lag_features_sales(df_combined)
df_combined = switch_type(df_combined)

# 6. 切分
test_final = df_combined[df_combined['date'] >= '2017-08-16'].copy()

in add_date
join oil
join_stores
join_holidays
join_payday
rename
join_earthquake
time_features
join_transactions
lag_features_sales
switch_type
in add_date
join oil
join_stores
join_holidays
join_payday
rename
join_earthquake
time_features
lag_features_sales
switch_type


In [139]:
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [140]:
# 确保日期是 datetime 格式
train['date'] = pd.to_datetime(train['date'])

# 1. 先对整个数据集按日期排序
train_sorted = train.sort_values(by='date').reset_index(drop=True)
print("数据集已完成全局日期排序。")

# 2. 划分自变量 X 和因变量 y
# 这里的 y 使用 sales 列
y = train_sorted['sales']

# X 包含除了 sales 以外的所有原始特征（包括 date，稍后用于划分）
# 同时去掉 id 和 store_nbr 等标识符（如果你之前决定不使用它们）
X = train_sorted.drop(columns=['sales', 'id', 'store_nbr'])

# 3. 计算切分点索引 (80%)
split_index = int(len(train_sorted) * 0.8)

# 划分训练集 (前80%)
X_train = X.iloc[:split_index]
y_train = y.iloc[:split_index]

# 划分测试集/验证集 (后20%)
X_test = X.iloc[split_index:]
y_test = y.iloc[split_index:]

# 对 y 进行 Log1p 转换（时间序列预测的常用技巧）
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

print(f"训练集大小: {X_train.shape}, 测试集大小: {X_test.shape}")
print(f"训练集日期范围: {X_train['date'].min()} 到 {X_train['date'].max()}")
print(f"测试集日期范围: {X_test['date'].min()} 到 {X_test['date'].max()}")

数据集已完成全局日期排序。
训练集大小: (2406624, 25), 测试集大小: (601656, 25)
训练集日期范围: 2013-01-01 00:00:00 到 2016-09-12 00:00:00
测试集日期范围: 2016-09-12 00:00:00 到 2017-08-15 00:00:00


In [141]:
# Preprocessing and pipeline（将特征工程处理放入一个管道中，避免数据溢出）
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)
numeric_feats = [
                 "dcoilwtico",
                 "onpromotion",
                 "day",
                 "day_of_week",
                 "month",
                 "year",
                 "lag_16_sales",
                 "lag_21_sales",
                 "lag_28_sales",
                 "rolling_mean_7_sales",
                 "lag_16_trans",
                 "lag_21_trans",
                 "lag_28_trans",
                 "rolling_mean_7_trans"
                 ]  # to apply imputation and scaling
categorical_feats = ["family", "city", "state", "store_group" ]  # to apply one-hot encoding
ordinal_feats = ["store_type"]  # to apply ordinal encoding
passthrough_features = ['is_payday', 'if_holiday', 'is_earthquake_period', 'is_weekend']
drop_features = ['date']

ct = ColumnTransformer(
    [
        ("numeric_feature", make_pipeline(StandardScaler()), numeric_feats),
        ("onehot", OneHotEncoder(sparse_output=False), categorical_feats),
        ("ordinal", OrdinalEncoder(categories=[['E', 'C', 'B', 'D', 'A']]), ordinal_feats),
        ("pass","passthrough",passthrough_features),
        ("drop","drop",drop_features)
    ]
)

X_train_transformed = ct.fit_transform(X_train)
X_test_transformed = ct.transform(X_test)
column_names = ct.get_feature_names_out()
X_train_transformed = pd.DataFrame(X_train_transformed, columns=column_names)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=column_names)
print("数据处理完成！")
print(f"训练集特征形状: {X_train_transformed.shape}")


数据处理完成！
训练集特征形状: (2406624, 107)


In [ ]:
# 预测后：模型输出的结果也是对数空间的，你需要用 np.expm1(y_pred)（即计算 $e^x - 1$）将其还原为真实的销量数值。
# import numpy as np

# # 1. 假设你已经排好序并分出了原始的 y_train_split 和 y_test_split
# # 2. 对 y 进行变换
# y_train_log = np.log1p(y_train_split)
# y_test_log = np.log1p(y_test_split)

# # 3. 训练模型（用变换后的 y）
# # model.fit(X_train_pca, y_train_log)

# # 4. 【重要】预测出来后，记得还原！
# # 因为模型输出的是对数，你需要把它变回真实的销量数值
# y_pred_log = model.predict(X_test_pca)
# y_final_prediction = np.expm1(y_pred_log) # expm1 是 log1p 的逆运算

In [142]:
# 如果内存允许，转为普通的 numpy 数组
X_train_final = X_train_transformed.astype('float32')
X_test_final = X_test_transformed.astype('float32')

In [143]:
X_train_final.head()

,numeric_feature__dcoilwtico,numeric_feature__onpromotion,numeric_feature__day,numeric_feature__day_of_week,numeric_feature__month,numeric_feature__year,numeric_feature__lag_16_sales,numeric_feature__lag_21_sales,numeric_feature__lag_28_sales,numeric_feature__rolling_mean_7_sales,...,onehot__store_group_13,onehot__store_group_14,onehot__store_group_15,onehot__store_group_16,onehot__store_group_17,ordinal__store_type,pass__is_payday,pass__if_holiday,pass__is_earthquake_period,pass__is_weekend
0,1.037906,-0.153992,-1.662543,-1.001087,-1.545136,-1.283054,-0.317491,-0.316659,-0.315489,-0.334191,...,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,1.037906,-0.153992,-1.662543,-1.001087,-1.545136,-1.283054,-0.317491,-0.316659,-0.315489,-0.334191,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2,1.037906,-0.153992,-1.662543,-1.001087,-1.545136,-1.283054,-0.317491,-0.316659,-0.315489,-0.334191,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,1.037906,-0.153992,-1.662543,-1.001087,-1.545136,-1.283054,-0.317491,-0.316659,-0.315489,-0.334191,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,1.037906,-0.153992,-1.662543,-1.001087,-1.545136,-1.283054,-0.317491,-0.316659,-0.315489,-0.334191,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [144]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_final)
X_test_pca = pca.transform(X_test_final)